# MongoDB - Lab

## Introduction

In this lesson, we'll get some hands-on experience with MongoDB!

## Objectives
You will be able to: 

- Create a MongoDB database   
- Insert data into a MongoDB database   
- Read data from a MongoDB database   
- Update data in a MongoDB database   

## Getting Started

To begin this lab, make sure that you start up the mongoDB server in your terminal first! **You must do this lab locally on your computer, not in Learn.**


## Connecting to the MongoDB Database

In the cell below, import the appropriate library and connect to the mongoDB server. Create a new database called `'lab_db'`.

In [24]:
import pymongo

myclient = pymongo.MongoClient('mongodb://localhost:27017')
mydb = myclient['lab_db']

## Creating a Collection

Now, create a collection called `'lab_collection'` inside `'lab_db'`.

In [25]:
mycollection = mydb['lab_collection']

## Adding Some Data

Now, we're going to add some example records into our database. In the cells below, create dictionary representations of the following customer records:


|     Name     |            Email           |  Mailing_Address  | Balance |                         Notes                         |
|:------------:|:--------------------------:|:-----------------:|:-------:|:-----------------------------------------------------:|
|  John Smith  |    j.smith@thesmiths.com   | 123 mulberry lane |   0.0   |    Called technical support, issue not yet resolved   |
|  Jane Smith  |  jane_smith@thesmiths.com  |         Null          |  25.00  |                   Null                                    |
|  Adam Enbar  | adam@theflatironschool.com |    11 Broadway    |  14.99  |           Set up on recurring billing cycle           |
| Avi Flombaum |  avi@theflatironschool.com |    11 Broadway    |   0.0   |                   Null                                    |
|   Steven S.  |     steven.s@gmail.net     |         Null          |  -20.23 | Refunded for overpayment due to price match guarantee |


Your first challenge is to take all of the data in the table above and create the corresponding documents and add then to our mongo database. Note that fields that contain 'Null' should not be included in the document (recall that since mongo is schema-less, each document can be different). 

Create the documents from the table listed above, and then use `insert_many()` to insert them into the collection.

In [26]:
customer_1 = {'Name': 'John Smith', 'Email': 'j.smith@thesmiths.com', 'Mailing_Address': '123 mulberry lane', 'Balance': 0.0, 'Notes': 'Called technical support, issue not yet resolved'}
customer_2 = {'Name': 'Jane Smith', 'Email': 'jane.smith@thesmiths.com', 'Mailing_Address': None, 'Balance': 25.00, 'Notes': None}
customer_3 = {'Name': 'Adam Enbar', 'Email': 'adam@theflatironschool.com', 'Mailing_Address': '11 Broadway', 'Balance': 14.99, 'Notes': 'Set up on recurring billing cycle'}
customer_4 = {'Name': 'Avi Flombaum', 'Email': 'avi@theflatironschool.com', 'Mailing_Address': '11 Broadway', 'Balance': 0.0, 'Notes': None}
customer_5 = {'Name': 'Steven S', 'Email': 'steven.s@gmail.net', 'Mailing_Address': None, 'Balance': -20.23, 'Notes': 'Refunded for overpayment due to price match guarantee'}

all_records = [customer_1, customer_2, customer_3, customer_4, customer_5]

insertion_results = mycollection.insert_many(all_records)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 5f7123ca357849d84986a6db, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

Now, access the appropriate attribute from the results object returned from the insertion to see the unique IDs for each record inserted, so that we can confirm each were inserted correctly. 

In [ ]:
insertion_results.inserted_ids

## Querying and Filtering

In the cell below, return the name and email address for every customer record. Then, print every item from the query to show that it worked correctly. 

In [ ]:
query_1 = mycollection.find({}, {'Name' : 1, 'Email' : 1})

for item in query_1:
    print(item)


Great! Now, let's write a query that gets an individual record based on a stored key-value pair a document contains. 

In the cell below, write a query to get the record for `'John Smith'` by using his name. Then, print the results of the query to demonstrate that it worked correctly.  

In [ ]:
query_2 = mycollection.find({}, {'Name' : 'John Smith'})

for item in query_2:
    print(item)


Great! Now, write a query to get the names, email addresses, and balances for customers that have a balance greater than 0. Use a modifier to do this. 

**_HINT_**: In the query below, you'll be passing in two separate dictionaries. The first one defines the logic of the query, while the second tells which fields we want returned. 

In [28]:
query_3 = mycollection.find({}, {'Name' : 1, 'Email' : 1, 'Balance' : {"$gt" : 0}})

for item in query_3:
    print(item)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 5f7123ca357849d84986a6db, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

## Updating a Record

Now, let's update some records. In the cell below. set the mailing address for `'John Smith'` to `'367 55th St., apt 2A'`.

In [29]:
record_to_update_1 = {'Name' : 'John Smith'}
update_1 = {'$set' : {'Mailing_Address' : '367 55th St., apt 2A'}}


Now, write a query to check that the update worked for this document in the cell below:  

In [31]:
query_4 = mycollection.find({}, {'Name' : 'John Smith'})
for item in query_4:
    print(item)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 5f7123ca357849d84986a6db, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

Now, let's assume that we want to add birthdays for each customer record. Consider the following table:

|     Name     |  Birthday  |
|:------------:|:----------:|
|  John Smith  | 02/20/1986 |
|  Jane Smith  | 07/07/1983 |
|  Adam Enbar  | 12/02/1982 |
| Avi Flombaum | 04/17/1983 |
|   Steven S.  | 08/30/1991 |

We want to add birthdays for each person, but we want to do so in a way where we don't have to do the same repetitive task over and over again. This seems like a good opportunity to write a function to handle some of the logic for us!

In the cell below:

* Store the names in the `names_list` variable as a list.
* Store the birthdays in the `birthdays_list` variable as a list.
* Write a function that takes in the two different lists, and updates each record by adding in the appropriate key-value pair containing that user's birthday.

**_Hint:_** There are several ways that you could write this, depending on whether you want to use the `update_one()` method or the `update_many()` method. See if you can figure out both approaches!

In [ ]:
names_list = None
birthdays_list = None

def update_birthdays(names, birthdays):
    pass
        
update_birthdays(names_list, birthdays_list)

Now, write a query to check your work and see that the birthdays were added correctly.

Great! It looks like the birthdays have been successfully added to every record correctly!

## Summary

In this lesson, we got some hands-on practice working with MongoDB!